## Les matrices impliquées

In [1]:
import openturns as ot

La matrice B.

In [2]:
parameterCovariance = ot.CovarianceMatrix([[ 4900, 0, 0 ], [ 0, 62500, 0 ], [ 0, 0, 0.64 ]])
parameterCovariance

class=CovarianceMatrix dimension=3 implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[4900,0,0,0,62500,0,0,0,0.64]

La matrice R.

In [3]:
size = 5
errorCovariance = ot.CovarianceMatrix(size)
for i in range(size):
    errorCovariance[i,i] = 100.
errorCovariance

class=CovarianceMatrix dimension=5 implementation=class=MatrixImplementation name=Unnamed rows=5 columns=5 values=[100,0,0,0,0,0,100,0,0,0,0,0,100,0,0,0,0,0,100,0,0,0,0,0,100]

La matrice J.

In [4]:
gradientObservations = ot.Matrix([[ 1, 0.297234, 77.4648 ], [ 1, 0.390046, 94.2324 ], [ 1, 0.0729565, 21.9462 ], [ 1, 0.0180369, 5.58539 ], [ 1, 0.176632, 50.0072 ]])
gradientObservations

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=5 columns=3 values=[1,1,1,1,1,0.297234,0.390046,0.0729565,0.0180369,0.176632,77.4648,94.2324,21.9462,5.58539,50.0072]

## Calcul de la matrice de Kalman

In [13]:
def computeMatrixInverse(A):
    '''Compute the inverse of a matrix'''
    p = A.getNbRows()
    I = ot.IdentityMatrix(p)
    invA = A.solveLinearSystem(I)
    return invA

In [14]:
def computeKalmanMatrix(parameterCovariance,errorCovariance,gradientObservations):
    invB = computeMatrixInverse(parameterCovariance)
    invR = computeMatrixInverse(errorCovariance)
    C = gradientObservations.transpose() * invR
    invA = invB + C * gradientObservations
    K = invA.solveLinearSystem(C)
    return K

In [15]:
K = computeKalmanMatrix(parameterCovariance,errorCovariance,gradientObservations)
K

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=3 columns=5 values=[-0.055002,-1.07739,0.00924803,-0.149205,4.09935,-0.00868822,0.429984,-1.1273,-0.000329947,0.596194,1.15781,-0.012433,0.164673,-3.09501,0.0125532]

In [31]:
def computeCovariance(parameterCovariance,errorCovariance,gradientObservations):
    K = computeKalmanMatrix(parameterCovariance,errorCovariance,gradientObservations)
    parameterDimension = parameterCovariance.getNbRows()
    IB = ot.IdentityMatrix(parameterDimension)
    L = IB - K * gradientObservations
    B = ot.CovarianceMatrix(parameterCovariance)
    R = ot.CovarianceMatrix(errorCovariance)
    covarianceThetaStar = K * R * K.transpose() + L * B * L.transpose()
    return covarianceThetaStar

In [32]:
covarianceThetaStar = computeCovariance(parameterCovariance,errorCovariance,gradientObservations)
covarianceThetaStar

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[65.4421,208.448,-1.71562,208.448,20583.9,-83.0626,-1.71562,-83.0626,0.352798]

In [34]:
invB = computeMatrixInverse(parameterCovariance)
invR = computeMatrixInverse(errorCovariance)
C = gradientObservations.transpose() * invR
invA = invB + C * gradientObservations
computeMatrixInverse(invA)

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[65.4421,208.448,-1.71562,208.448,20583.9,-83.0626,-1.71562,-83.0626,0.352798]